# Importo librerías

In [ ]:
import eikon as ek
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
from textblob import TextBlob
import datetime
from datetime import time
import warnings
warnings.filterwarnings("ignore")

In [ ]:
ek.set_app_id('THEWOODBRIDGECOMPANY.PYPY')

# Guardo titulares e IDs de noticias del último año

Guardo titulares de noticias del último año, la API limita a 100 noticias pero con un bucle puedo concatenar más:

In [ ]:
deb_abre, fin_abre = '2017-12-07T09:00:00', '2018-06-12T09:00:00'
deb_cierra, fin_cierra = '2017-12-07T16:55:00', '2018-06-12T16:55:00'

start_abre, end_abre = datetime.datetime.strptime(deb_abre, "%Y-%m-%dT%H:%M:%S"), \
                                    datetime.datetime.strptime(fin_abre, "%Y-%m-%dT%H:%M:%S")
start_cierra, end_cierra = datetime.datetime.strptime(deb_cierra, "%Y-%m-%dT%H:%M:%S"), \
                                    datetime.datetime.strptime(fin_cierra, "%Y-%m-%dT%H:%M:%S")

dif = int((end_abre-start_abre).total_seconds()/(3600*24)) ## time difference in days

fechas_iniciales = [(start_abre + datetime.timedelta(hours=24*x)).strftime("%Y-%m-%dT%H:%M:%S") for x in range(dif+1)]
fechas_finales = [(start_cierra + datetime.timedelta(hours=24*x)).strftime("%Y-%m-%dT%H:%M:%S") for x in range(dif+1)]

In [ ]:
df = pd.DataFrame()
for fecha, fecha_fin in zip(fechas_iniciales,fechas_finales): 
    aux = ek.get_news_headlines('R:IBM.N AND Language:LEN', date_from = fecha, date_to = fecha_fin, count=20)
    df = pd.concat([df, aux])
df.head()

In [ ]:
elementos = len(df.index)
print(u'Tengo un total de %s noticias que analizar' %elementos)

# Análisis de sentimientos de las noticias

Creo 3 columnas para guardar variables que generaré posteriormente:

In [ ]:
df['Polaridad'] = np.nan
df['Subjetividad'] = np.nan
df['Categorizacion'] = np.nan

Tenemos un dataframe con los titulares de las noticias y los ID que ha asignado Thomson Reuters a esas noticias. Con ese ID seremos capaces de acceder a las noticias en sí y analizarlas:

In [ ]:
for indice, noticiaID in enumerate(df['storyId'].values): # Itero por todas las filas del dataframe
    try:
        texto = ek.get_news_story(noticiaID) # Obtengo el texto de cada una de las noticias
        if texto:
            soup = BeautifulSoup(texto,"lxml") # Creo un objeto BeautifulSoup a partir de nuestro artículo HTML
            sents = TextBlob(soup.get_text()) # Paso el texto del artículo a TextBlob para que lo analice
            df['Polaridad'].iloc[indice] = sents.sentiment.polarity # Guardo la polaridad del sentimiento en el dataframe
            df['Subjetividad'].iloc[indice] = sents.sentiment.subjectivity # Guardo la subjetividad 0->objetiva 1->subjetiva
            if sents.sentiment.polarity >= 0.05: # Categorizo las polaridades -1->negativa 1->positiva
                score = 'Positivo'
            elif  -.05 < sents.sentiment.polarity < 0.05:
                score = 'Neutral'
            else:
                score = 'Negativo'
            df['Categorizacion'].iloc[indice] = score 
            if indice%20==0: # Pongo un contador para ver por donde va el tema
                print(u'Voy por el índice %s' %indice)
    except:
        pass
df.head(3)

# Analizo el impacto de las noticias sobre el precio (ceteris paribus)

Obtengo una serie temporal de precios en cada minuto, desde la fecha mínima de noticias. Ojo, como máximo devuelve 50 mil minutos por lo que tengo aproximadamente 6 meses de datos (va desde el presente hasta la fecha de inicio).

In [ ]:
"""minuto = pd.DataFrame()
for fecha, fecha_fin in zip(fechas_iniciales,fechas_finales): 
    aux = ek.get_timeseries(["IBM.N"], start_date = fecha, end_date = fecha_fin, interval='minute')
    minuto = pd.concat([minuto, aux])
minuto.tail()"""

In [ ]:
inicio = df['versionCreated'].min().replace(hour=0,minute=0,second=0,microsecond=0).strftime('%Y/%m/%d')
fin = df['versionCreated'].max().replace(hour=0,minute=0,second=0,microsecond=0).strftime('%Y/%m/%d')
minuto = ek.get_timeseries(["IBM.N"], start_date=inicio, interval="minute")
minuto.tail()

In [ ]:
min(minuto.index), max(minuto.index)

Defino variables con los precios en minutos posteriores:

In [ ]:
df['2Minutos'] = np.nan
df['5minutos'] = np.nan
df['10minutos'] = np.nan
df['30minutos'] = np.nan
df['1hora'] = np.nan
df.head(3)

Obtengo la variación en cada intervalo de tiempo como:
$$\triangle_{t\rightarrow t+x} = \Bigl ( \frac{Valor_{t+x}}{Valor_t} -1\Bigr) * 100$$

In [ ]:
for indice, fechaNoticia in enumerate(df['versionCreated'].values):
    sTime = df['versionCreated'][indice]
    sTime = sTime.replace(second=0,microsecond=0) # Quito segundos y micros para ir a nivel minuto
    try:
        t0 = minuto.iloc[minuto.index.get_loc(sTime),2] # Lo que vale al crearse la noticia
        df['2Minutos'][indice] = ((minuto.iloc[minuto.index.get_loc((sTime + datetime.timedelta(minutes=2))),3]/(t0)-1)*100)
        df['5minutos'][indice] = ((minuto.iloc[minuto.index.get_loc((sTime + datetime.timedelta(minutes=5))),3]/(t0)-1)*100)
        df['10minutos'][indice] = ((minuto.iloc[minuto.index.get_loc((sTime + datetime.timedelta(minutes=10))),3]/(t0)-1)*100) 
        df['30minutos'][indice] = ((minuto.iloc[minuto.index.get_loc((sTime + datetime.timedelta(minutes=30))),3]/(t0)-1)*100)
        df['1hora'][indice] = ((minuto.iloc[minuto.index.get_loc((sTime + datetime.timedelta(minutes=60))),3]/(t0)-1)*100)
    except:
        pass
df.head(3)

In [ ]:
df.isnull().sum()

In [ ]:
df = df.dropna()
df.head(3)

In [ ]:
grouped = df.groupby(['Categorizacion']).mean()
grouped

Las noticias negativas hacen bajar las acciones de IBM de media en los 10 minutos siguientes un 0,14 por ciento, mientras que las buenas la hacen bajar un 0,011 por ciento. Ojo, esto está sesgado a las acciones de IBM pero lo que se deduce es que a muy corto plazo las noticias impactan en el precio de las acciones.

# Visualizaciones

In [ ]:
df = df[~df.index.duplicated()]

In [ ]:
positivos = df.loc[df['Categorizacion']=='Positivo'].groupby(df['versionCreated'].dt.\
                            strftime('%Y-%m-%d'))['Categorizacion'].count().sort_index().reset_index()
negativos = df.loc[df['Categorizacion']=='Negativo'].groupby(df['versionCreated'].dt.\
                            strftime('%Y-%m-%d'))['Categorizacion'].count().sort_index().reset_index()
neutrales = df.loc[df['Categorizacion']=='Neutral'].groupby(df['versionCreated'].dt.\
                            strftime('%Y-%m-%d'))['Categorizacion'].count().sort_index().reset_index()

In [ ]:
minuto_represento = minuto.reset_index()
minuto_represento = minuto_represento[['Date','OPEN']].groupby(minuto_represento['Date'].\
                                                dt.strftime('%Y-%m-%d')).mean().reset_index()

In [ ]:
import plotly
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
import plotly.graph_objs as go
init_notebook_mode(connected=True)
# Create traces
trace0 = go.Scatter(
    x = positivos['versionCreated'],
    y = positivos['Categorizacion'],
    line = dict(color = 'green'),
    mode = 'lines+markers',
    name = 'Noticias positivas',
    yaxis='y2'
)
trace1 = go.Scatter(
    x = negativos['versionCreated'],
    y = negativos['Categorizacion'],
    line = dict(color = 'red'),
    mode = 'lines+markers',
    name = 'Noticias negativas',
    yaxis='y2'
)
trace2 = go.Scatter(
    x = neutrales['versionCreated'],
    y = neutrales['Categorizacion'],
    line = dict(color = 'orange'),
    mode = 'lines+markers',
    name = 'Noticias neutrales',
    yaxis='y2'
)
trace3 = go.Scatter(
    x = minuto_represento['Date'],
    y = minuto_represento['OPEN'],
    line=go.Line(shape='hv', color='black'),
    mode = 'lines',
    fill='tonexty',
    name = 'Precio de las acciones'
)
datos = [trace0, trace1, trace2, trace3]

layout = dict(title = u'Análisis de sentimiento de noticias sobre IBM',
                xaxis = dict(title = u'Fecha'),
                yaxis = dict(title = u'Precio de las acciones'),
                yaxis2=dict(
                    title=u'Número de noticias',
                    overlaying='y',
                    side='right'
                )
              )

fig = dict(data=datos, layout=layout)
iplot(fig, filename='analisis-noticias')

In [ ]:
df.to_csv('analisis_noticias_fichero.csv', encoding='utf-8', sep=';', index=False)